# Кластеризація методом КРАБ.
Черних Максим

# Завдання
1. Реалізувати алгоритм та застосувати його. Розділення кластерів проводити по найдовшому ребру.
2. Вивести таблицю з кроками об'єднання.
3. Оцінити якість кластеризації за критерієм F. Вивести значення всіх чотирьох характеристик.
4. Зробити висновок.

# Реалізація алгоритму кластерізації методом КРАБ.

Iмпортування необхідних для роботи бібліотек

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Завантаження датасету смертності від травм

In [ ]:
pd.read_excel(f"InjuryDeath.xlsx",)

,age,inj_dth,childyrs,age_mom,lbw,educ_mom,income,illegit,oth_chld,race_mom,pnclate
0,0,0,28.294319,24,1,15,3,1,0,0,0
1,0,0,0.091718,24,1,11,2,1,2,0,1
2,1,0,20.000000,24,1,15,3,0,3,0,0
3,0,0,10.000000,30,1,16,1,1,2,1,0
4,0,0,10.000000,30,1,12,3,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
15181,4,0,18.535250,19,1,12,2,1,0,1,1
15182,0,0,31.711157,24,0,11,5,1,1,0,1
15183,2,0,23.915127,30,0,16,4,0,0,0,1
15184,0,0,15.544147,29,0,16,1,1,1,1,1


Нормування даних

In [ ]:
def df_dataset():
  df = pd.read_excel(f"InjuryDeath.xlsx",)
  df = df.dropna()
  for col_name in df.columns.tolist():
        df[col_name] = (df[col_name] - np.min(df[col_name])) / (np.max(df[col_name]) - np.min(df[col_name]))

  points = df[['childyrs', 'oth_chld', 'lbw']].values

  return df, points

df_dataset()[0]

,age,inj_dth,childyrs,age_mom,lbw,educ_mom,income,illegit,oth_chld,race_mom,pnclate
0,0.00,0.0,0.004531,0.454545,1.0,0.8,0.50,1.0,0.00,0.0,0.0
1,0.00,0.0,0.000015,0.454545,1.0,0.0,0.25,1.0,0.50,0.0,1.0
2,0.25,0.0,0.003203,0.454545,1.0,0.8,0.50,0.0,0.75,0.0,0.0
3,0.00,0.0,0.001601,1.000000,1.0,1.0,0.00,1.0,0.50,1.0,0.0
4,0.00,0.0,0.001601,1.000000,1.0,0.2,0.50,1.0,0.00,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
15181,1.00,0.0,0.002968,0.000000,1.0,0.2,0.25,1.0,0.00,1.0,1.0
15182,0.00,0.0,0.005078,0.454545,0.0,0.0,1.00,1.0,0.25,0.0,1.0
15183,0.50,0.0,0.003830,1.000000,0.0,1.0,0.75,0.0,0.00,0.0,1.0
15184,0.00,0.0,0.002489,0.909091,0.0,1.0,0.00,1.0,0.25,1.0,1.0


Бачимо, що у даному датасетi даних дуже багато, а саме 15 тисяч рядкiв

In [ ]:
def df_dataset():
  df = pd.read_excel(f"InjuryDeath.xlsx", usecols = ['childyrs', 'oth_chld', 'lbw'])
  df = df.dropna()
  columns = df.columns.tolist()
  for col in df.columns.tolist():
        df[col] = (df[col] - np.min(df[col])) / (np.max(df[col]) - np.min(df[col]))

  return df, df.columns.tolist(), df.values

У ходi експериментiв з'ясувалося, що при 15к даних алгоритм працюе досить довго, тому аби краще його реалiзувати та проаналiзувати декiлька варiантiв отриманих данних, було вирiшено обрати випадковi 1000 об'ектiв з датасете

In [ ]:
data, col, points = df_dataset()

random_choice = np.random.choice(list(range(data.shape[0])), 1000)

index = []

for i in random_choice:
    index.append(i)


df = pd.DataFrame(data, index=index, columns=col)

df

,childyrs,lbw,oth_chld
8085,0.003203,0.0,0.75
10524,0.004801,0.0,0.50
8163,0.001602,1.0,0.75
9161,0.006687,1.0,0.50
12037,0.002614,0.0,0.25
...,...,...,...
6702,0.001601,0.0,0.50
5600,0.001601,1.0,0.75
6528,0.057548,0.0,0.00
14807,0.008001,1.0,0.75


Обираємо метрику евклідовоіі  відстані та функцію знаходження мінімального індексу

In [ ]:
def euclidean(p1, p2):
    dist = 0
    for i in range(len(p1)):
        dist += (p1[i] - p2[i]) ** 2
    return dist ** 0.5

def connecting(point, array):
    min_dist = 1000000
    for i in range(array.shape[0]):
        dist = euclidean(point, array[i])
        if dist < min_dist:
            min_dist = dist
            min_index = i
    return min_index

Реалiзацiя алгоритму

In [ ]:
norm_df = df
count_clusters = 5
cols = norm_df.columns.tolist()

array = norm_df.values

points = [] 
indices_1 = [] 
indices_2 = []  
rand_index = np.random.randint(low=0, high=norm_df.shape[0] - 1)
indices_1.append(rand_index)
actual_array = copy.deepcopy(array)
points.append(actual_array[rand_index])
actual_array = np.delete(arr=actual_array, obj=rand_index, axis=0)
dists = []
while len(points) < norm_df.shape[0]:
    index = connecting(points[-1], actual_array)
    index_from_init = np.where(np.all(array == actual_array[index], axis=1))[0][0]
    indices_1.append(index_from_init)
    indices_2.append(index_from_init)
    points.append(actual_array[index])
    dists.append(euclidean(points[-1], points[-2]))
    actual_array = np.delete(arr=actual_array, obj=index, axis=0)
del indices_1[-1]
df = pd.DataFrame({'1': indices_1, '2': indices_2, 'Dist': dists})
print()
print('Clusters: ', count_clusters)
df


Clusters:  5


,1,2,Dist
0,153,153,0.000000
1,153,660,4.007860
2,660,548,16.007826
3,548,226,9.003481
4,226,875,34.005016
...,...,...,...
994,706,441,8.066135
995,441,210,1.500000
996,210,757,5.000000
997,757,214,3.092333


Знаходження найдовших ребер графу

In [ ]:
df = df.reset_index()
df = df.sort_values(by=['Dist'], ascending=False)
longest_dists_indices = list(df['index'].values)[:count_clusters]
longest_dists_indices.sort()
true_false = []
df = df.sort_values(by=['index'])
df = df.drop(columns=['index'])

Створення кластерiв

In [ ]:
clusters = []
clusters_dists = []
if count_clusters > 2:
    for i in range(count_clusters):
        if i == 0:
            clusters.append(points[:longest_dists_indices[i] + 1])
            clusters_dists.append(dists[:longest_dists_indices[i]])
        elif i == count_clusters - 1:
            clusters.append(points[longest_dists_indices[i] + 1:])
            clusters_dists.append(dists[longest_dists_indices[i] + 1:])
        else:
            clusters.append(points[longest_dists_indices[i - 1] + 1:longest_dists_indices[i] + 1])
            clusters_dists.append(dists[longest_dists_indices[i - 1] + 1:longest_dists_indices[i]])
else:
    clusters.append(points[:longest_dists_indices[i] + 1])
    clusters.append(points[longest_dists_indices[i] + 1:])
    clusters_dists.append(dists[:longest_dists_indices[i]])
    clusters_dists.append(dists[longest_dists_indices[i] + 1:])
final_df = []
clusters_list = []
for i in range(count_clusters):
    for j in range(len(clusters[i])):
        final_df.append(list(clusters[i][j]))
        clusters_list.append(i + 1)
final_df = pd.DataFrame(final_df, columns=cols)
final_df['Clusters'] = clusters_list

In [ ]:
final_df

,index,childyrs,lbw,oth_chld,Clusters
0,421.0,0.004584,0.0,0.50,1
1,421.0,0.004584,0.0,0.50,1
2,425.0,0.025519,0.0,0.75,1
3,409.0,0.003203,0.0,0.25,1
4,400.0,0.015954,0.0,0.50,1
...,...,...,...,...,...
710,15161.0,0.001601,1.0,0.50,5
711,15162.0,0.002806,0.0,1.00,5
712,15167.0,0.001748,0.0,1.00,5
713,15170.0,0.006402,0.0,0.25,5


ВИведення результатiв (F -  критерiй)

In [ ]:
p = []
h_value = 1
for i in range(len(clusters)):
    p.append(np.mean(clusters_dists[i]))
    h_value *= (len(clusters[i]) / array.shape[0])

for i in range(len(p)):
    print('Cluster', i + 1)
    print('p-value: ', p[i])
p_value = np.mean(pd.DataFrame(p).dropna())

Cluster 1
p-value:  14.091069519868114
Cluster 2
p-value:  14.515662914837657
Cluster 3
p-value:  16.50474119614887
Cluster 4
p-value:  15.341149021614346
Cluster 5
p-value:  8.80564980960325


In [ ]:
p_value[0]

13.851654492414449

In [ ]:
d = []
for i in longest_dists_indices:
    d.append(dists[i])
d_value = np.mean(d)

d_value

168.40258981433746

In [ ]:
ind_max = list(dists).index(np.max(dists))
lambda_value = dists[ind_max - 1] / dists[ind_max]

lambda_value

0.03348677100114777

In [ ]:
h_value *= count_clusters ** count_clusters

h_value

0.022794000075

In [ ]:
f_value = np.log((d_value * h_value) / (lamda_value * p_value))

f_value[0]

2.1132996785728357

# Висновки

У данiй роботi було розроблено алгоритм кластерiзацii сiмейства КРАБ. Для пришвидшення розробки та полегшення аналiзу результатiв убло вирiшено зменшити датасет з 15к рядкiв до 1000. 

Критерiй якостi вийшов досить позитивним для нас (при експериментах та пiд час розробки вiн був ближче до 0 та i зовсiм вiд'емним. Також було обрано 5 кластерiв, так як кiлькiсть даних дуже велика, а судячи по результатам попередньоii лабораторноii работи, було видно, що саме 5 кластерiв з евклiдовою метрикою найкраще справляються з кластерiзацiею на моему датасетi.